In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr

In [2]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("API key not found")
else:
    print("API key found")

API key found


In [151]:
openai = OpenAI()
model = 'gpt-4.1-mini'
MODEL = 'gpt-4.1-mini'

In [4]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
def chat(message, history):
    system_prompt = [{'role': 'system', 'content': system_message}]
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = system_prompt + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


In [60]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    ticket_price = ticket_prices.get(destination_city.lower(), "Unknown city")
    return f"The price of a ticket to {destination_city} is {ticket_price}"

In [61]:
get_ticket_price("Paris")

Tool called for city Paris


'The price of a ticket to Paris is $899'

In [136]:
ticket_price_tool = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [107]:
# ticket_price_tool = {
#     "type": "function",
#     "function": {
#         "name": "get_ticket_price",
#         "description": "Returns the ticket price for a given destination city.",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "destination_city": {
#                     "type": "string",
#                     "description": "Name of the destination city to check the ticket price."
#                 }
#             },
#             "required": ["destination_city"]
#         }
#     }
# }

In [137]:
tools = [{"type": "function", "function": ticket_price_tool}]

In [138]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [139]:
import json

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [140]:
def chat_message(message, history):
    system_prompt = [{'role': 'system', 'content': system_message}]
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = system_prompt + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=='tool_calls':
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        # for message in messages:
        #     print(message)

        response = openai.chat.completions.create(model=model, messages=messages)
    
    return response.choices[0].message.content

In [141]:
gr.ChatInterface(fn=chat_message, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7902
* To create a public link, set `share=True` in `launch()`.


Tool called for city paris


In [164]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses


In [ ]:
def chat_message(message, history):
    system_prompt = [{'role': 'system', 'content': system_message}]
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = system_prompt + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=model, messages=messages)
    
    return response.choices[0].message.content

In [167]:
gr.ChatInterface(fn=chat_message, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7917
* To create a public link, set `share=True` in `launch()`.


Tool called for city London


In [170]:
def chat_message(message, history):
    system_prompt = [{'role': 'system', 'content': system_message}]
    history = [{'role': h['role'], 'content': h['content']} for h in history]
    messages = system_prompt + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)

    while response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=model, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat_message, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7919
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
Tool called for city Paris
Tool called for city Tokyo
Tool called for city Berlin
Tool called for city Tokyo
Tool called for city Berlin


In [187]:
import sqlite3

DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [189]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [190]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'No price data available for this city'

In [192]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [193]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [199]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT city, price FROM prices")
    rows = cursor.fetchall()

for row in rows:
    print(row)

('london', 799.0)
('paris', 899.0)
('tokyo', 1420.0)
('sydney', 2999.0)
('bangalore', 2100.0)


In [196]:
set_ticket_price("Bangalore", 2100)

In [201]:
get_ticket_price("Berlin")

DATABASE TOOL CALLED: Getting price for Berlin


'No price data available for this city'

In [200]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7921
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for bangalore


In [202]:
import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
        model = "dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(image_data)


NameError: name 'image_response' is not defined